Geolocalizador de coordenadas geográficas de endereços. As vezes, temos um dataframe com uma ou mais colunas de localização, entretanto para criar gráficos com geolocalizadores é necessário adicionar as latitudes e longitudes, para isso esse código serve. Uma recomendação é criar um coluna com todas as informações relevantes ao problema, por exemplo: no dataframe abaixo, a grande maioria dos pontos foi encontrada em Maceió, porém outros foram espalhadas em todo o Brasil, provavelmente devido a nomes de localidades iguais. Para resolver esse problema o indicado é ter uma coluna com todas as infromações que são utéis para achar o endereço. Eu utilizei municipio e bairro, mas vc pode adicionar uma string manual que ajude nessa busca.
Referência: 'https://www.youtube.com/watch?v=asSNKH_ee0M&ab_channel=Programa%C3%A7%C3%A3oDin%C3%A2mica'

In [14]:
import pandas as pd
url = 'https://dados.al.gov.br/catalogo/dataset/821e5f37-0be4-4a5e-948c-59d95e538773/resource/ec77b1d0-684d-48c5-9196-23ff216c9dfb/download/bairrosexcel2-1.csv'
df = pd.read_csv(url, encoding='ISO-8859-1', delimiter = ';')  # Especificando a codificação correta
df


,Bairro,Município,Populção Total(Senso 2010),Pop. De Homens (Senso 2010),Pop. De Mulheres (Senso 2010)
0,Benedito Bentes,Maceió,88084,42312,45772
1,Mutange,Maceió,2632,1248,1384
2,Ouro Preto (Maceió),Maceió,6224,2960,3264
3,Pajuçara,Maceió,3711,1675,2036
4,Barro Duro,Maceió,14431,6758,7673
...,...,...,...,...,...
116,Baixa Grande,Arapiraca,4691,2227,2464
117,João Paulo II,Arapiraca,1509,717,792
118,Santa Esmeralda,Arapiraca,4808,2273,2535
119,Manoel Teles,Arapiraca,8255,3901,4354


In [15]:
df['procurar'] = df['Município'].str.cat(df['Bairro'], sep=', ')

In [16]:
df['procurar']

0          Maceió, Benedito Bentes
1                  Maceió, Mutange
2      Maceió, Ouro Preto (Maceió)
3                 Maceió, Pajuçara
4               Maceió, Barro Duro
                  ...             
116        Arapiraca, Baixa Grande
117       Arapiraca, João Paulo II
118     Arapiraca, Santa Esmeralda
119        Arapiraca, Manoel Teles
120        Arapiraca, Massaranduba
Name: procurar, Length: 121, dtype: object

In [17]:
df['Município'].unique()

array(['Maceió', 'Barra de São Miguel', 'Penedo', 'Palmeira dos Índios',
       'Arapiraca'], dtype=object)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Bairro                         121 non-null    object
 1   Município                      121 non-null    object
 2   Populção Total(Senso 2010)     121 non-null    int64 
 3   Pop. De Homens (Senso 2010)    121 non-null    int64 
 4   Pop. De Mulheres (Senso 2010)  121 non-null    int64 
 5   procurar                       121 non-null    object
dtypes: int64(3), object(3)
memory usage: 5.8+ KB


In [19]:
from geopy.geocoders import Nominatim

In [20]:
locator = Nominatim(user_agent='MyGeocoder')
location = locator.geocode("Avenida Braulino Santos, 571")
location.point

Point(-14.8647567, -40.8216615, 0.0)

In [21]:
from geopy.extra.rate_limiter import RateLimiter

In [22]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [23]:
df['raw'] = df['procurar'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Barra de São Miguel, Centro B. de São Miguel',), **{}).
Traceback (most recent call last):
  File "C:\Users\reneg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\reneg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1520.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1520.0_x64__qbz5n2kfra8p0\Lib\http

In [24]:
df['raw'][0]

Location(Benedito Bentes, Maceió, Região Geográfica Imediata de Maceió, Região Geográfica Intermediária de Maceió, Alagoas, Região Nordeste, Brasil, (-9.5561682, -35.7237673, 0.0))

In [25]:
df['raw'].isna().sum()

20

In [26]:
df.dropna(subset=['raw'], inplace=True)
df['raw'].isna().sum()


0

In [27]:
df['LATITUDE'] = df['raw'].apply(lambda loc: loc.latitude if loc else None)
df['LONGITUDE'] = df['raw'].apply(lambda loc: loc.longitude if loc else None)
df['POINT'] = df['raw'].apply(lambda loc: loc.point if loc else None)

In [31]:
df['raw']

0      (Benedito Bentes, Maceió, Região Geográfica Im...
1      (Mutange, Maceió, Região Geográfica Imediata d...
3      (Pajuçara, Maceió, Região Geográfica Imediata ...
4      (Barro Duro, Maceió, Região Geográfica Imediat...
5      (Pescaria, Maceió, Região Geográfica Imediata ...
                             ...                        
116    (Baixa Grande, Arapiraca, Região Geográfica Im...
117    (Escola João Paulo II, Rua São João, São José ...
118    (Santa Esmeralda, Arapiraca, Região Geográfica...
119    (Manoel Teles, Arapiraca, Região Geográfica Im...
120    (Massaranduba, Arapiraca, Região Geográfica Im...
Name: raw, Length: 101, dtype: object

In [39]:
import folium
mapa=folium.Map(
    location=[df['LATITUDE'].mean(), df['LONGITUDE'].mean()], 
    zoom_start=10)
mapa

import folium
from folium.plugins import MarkerCluster

# Função para criar pop-up com latitude e longitude
def create_popup(row):
    popup_text = f"Latitude: {row['LATITUDE']}<br>Longitude: {row['LONGITUDE']}"
    return folium.Popup(popup_text, max_width=400)

# Adicione os marcadores ao mapa com pop-ups
df.apply(lambda row: folium.Marker(
    location=[row['LATITUDE'], row['LONGITUDE']],
    popup=create_popup(row)
).add_to(mapa), axis=1)


# Exiba o mapa
mapa


In [40]:
df.to_csv('Bairro_maceio_coordendas.csv', index=False)